# 데이터 삽입

In [1]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {
            "title": "Kafka on the Shore",
            "author": "Haruki Murakami",
            "year": 2002,
            "genre": "fantasy",
        },
        {
            "title": "Norwegian Wood",
            "author": "Haruki Murakami",
            "year": 1987,
            "genre": "thriller",
        },
        {
            "title": "1Q84",
            "author": "Haruki Murakami",
            "year": 2009,
            "genre": "fantasy",
        },
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {
            "title": "Inception",
            "director": "Christopher Nolan",
            "year": 2010,
            "rating": 8.8,
            "genre": "fantasy",
        },
        {
            "title": "Interstellar",
            "director": "Christopher Nolan",
            "year": 2014,
            "rating": 8.6,
            "genre": "fantasy",
        },
        {
            "title": "The Dark Knight",
            "director": "Christopher Nolan",
            "year": 2008,
            "rating": 9.0,
            "genre": "fantasy",
        },
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()


In [2]:
insert_data()

Data inserted successfully


In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client.local  # 'local' 데이터베이스 사용

## 문제1

In [5]:
genre = 'fantasy'

In [6]:
books_collection = db.books
query = {"genre": genre}
projection = {"_id": 0, "title": 1, "author": 1}

books = books_collection.find(query, projection)
for book in books:
    print(book)

{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}


## 문제2

In [7]:
movies_collection = db.movies
pipeline = [
    {"$group": {"_id": "$director", "average_rating": {"$avg": "$rating"}}},
    {"$sort": {"average_rating": -1}}
]

results = movies_collection.aggregate(pipeline)
for result in results:
    print(result)

{'_id': 'Christopher Nolan', 'average_rating': 8.8}


## 문제3

In [8]:
user_id = 1

In [10]:
user_actions_collection = db.user_actions
query = {"user_id": user_id}
sort_criteria = [("timestamp", -1)]

actions = user_actions_collection.find(query).sort(sort_criteria).limit(5)
for action in actions:
    print(action)

{'_id': ObjectId('662e56af33c491230682b3f1'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662e55f4f515907c182527d0'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662e5680f515907c182527da'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662e5680f515907c182527d9'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}
{'_id': ObjectId('662e55f4f515907c182527cf'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}


## 문제4

In [11]:
books_collection = db.books
pipeline = [
    {"$group": {"_id": "$year", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

results = books_collection.aggregate(pipeline)
for result in results:
    print(result)

{'_id': 2002, 'count': 3}
{'_id': 2009, 'count': 3}
{'_id': 1987, 'count': 3}


## 문제5

In [12]:
from datetime import datetime

In [13]:
date = datetime(2023,4,10)
old_action = 'view'
new_action = 'seen'

In [14]:
user_actions_collection = db.user_actions
query = {"user_id": user_id, "action": old_action, "timestamp": {"$lt": date}}
update = {"$set": {"action": new_action}}

result = user_actions_collection.update_many(query, update)
print(f"Updated {result.modified_count} documents.")

Updated 0 documents.
